In [11]:
import os
import nest_asyncio
from dbgpt.agent import (
    AgentContext,
    AgentResource,
    GptsMemory,
    LLMConfig,
    ResourceLoader,
    ResourceType,
    UserProxyAgent,
)
from dbgpt.agent.expand.plugin_assistant_agent import PluginAssistantAgent
from dbgpt.agent.expand.summary_assistant_agent import SummaryAssistantAgent
from dbgpt.agent.plan import WrappedAWELLayoutManager
from dbgpt.agent.resource import PluginFileLoadClient
from dbgpt.configs.model_config import ROOT_PATH
from dbgpt.model.proxy import OpenAILLMClient

nest_asyncio.apply()
test_plugin_dir = os.path.join(ROOT_PATH, "examples/test_files/plugins")

In [12]:
# os.environ['OPENAI_API_KEY']="sk-x"
# os.environ['OPENAI_API_BASE']="https://proxy_url/v1"
# os.environ['SEARCH_ENGINE']="baidu"
# os.environ['BAIDU_COOKIE']="""your baidu cookie"""

llm_client = OpenAILLMClient(model_alias="gpt-3.5-turbo")
context: AgentContext = AgentContext(conv_id="test456", gpts_app_name="信息析助手")

default_memory = GptsMemory()

resource_loader = ResourceLoader()
plugin_file_loader = PluginFileLoadClient()
resource_loader.register_resource_api(plugin_file_loader)

plugin_resource = AgentResource(
    type=ResourceType.Plugin,
    name="test",
    value=test_plugin_dir,
)

tool_engineer = (
    await PluginAssistantAgent()
    .bind(context)
    .bind(LLMConfig(llm_client=llm_client))
    .bind(default_memory)
    .bind([plugin_resource])
    .bind(resource_loader)
    .build()
)
summarizer = (
    await SummaryAssistantAgent()
    .bind(context)
    .bind(default_memory)
    .bind(LLMConfig(llm_client=llm_client))
    .build()
)

manager = (
    await WrappedAWELLayoutManager()
    .bind(context)
    .bind(default_memory)
    .bind(LLMConfig(llm_client=llm_client))
    .build()
)
manager.hire([tool_engineer, summarizer])

user_proxy = await UserProxyAgent().bind(context).bind(default_memory).build()

await user_proxy.initiate_chat(
    recipient=manager,
    reviewer=user_proxy,
    message="查询成都今天天气",
    # message="查询今天的最新热点财经新闻",
)


--------------------------------------------------------------------------------
WrappedAWELLayoutManager (to LuBan)-[]:

"查询成都今天天气"

--------------------------------------------------------------------------------
un_stream ai response: {
  "tool_name": "baidu_search",
  "args": {
    "query": "成都今天天气"
  },
  "thought": "I have selected the 'baidu_search' tool with the query parameter set to '成都今天天气' to search for the weather in Chengdu today."
}
{'query': '成都今天天气'}

--------------------------------------------------------------------------------
LuBan (to Aristotle)-[gpt-3.5-turbo]:

"{\n  \"tool_name\": \"baidu_search\",\n  \"args\": {\n    \"query\": \"成都今天天气\"\n  },\n  \"thought\": \"I have selected the 'baidu_search' tool with the query parameter set to '成都今天天气' to search for the weather in Chengdu today.\"\n}"
>>>>>>>>LuBan Review info: 
Pass(None)
>>>>>>>>LuBan Action report: 
execution succeeded,
### [...天气预报一周_成都天气预报7天、15天、40天天查询_中国...](http://www.baidu.com/link?url=nSNTTnrx